In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

from myBuilding import runMyBEM
from runMyBuildingMC import getConstructions, getWeatherData, setConstructionType
from model.WallSimulation import convectionDOE2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
material_type = "Heavy"
materials = getConstructions("My", constructionFile = "energyPlus/My_Constructions.csv")
chosenMaterial = setConstructionType(materials, material_type)
chosenMaterial

In [ ]:
dt = 30
chosenZone = 12
chosenMonth = 9
data, climate_zones = getWeatherData()
dataSampled = data[(data["ClimateZone"] == chosenZone) & (data.index.month == chosenMonth)]
dataSampled = dataSampled.iloc[1:]
dataSampled = dataSampled.infer_objects(copy=False)
dataSampled = dataSampled.resample(f"{dt}s").interpolate()

In [ ]:
plt.figure()
plt.plot(dataSampled["Dry Bulb Temperature"][1:])

In [ ]:
floorTempAdjustment = np.mean([-3.5, -5])
hInterior = np.mean([1, 3])
alphaRoof = np.mean([0.6, 0.9])
windSpeed = np.mean([0, 6])
windSpeed = np.mean([0, 6])
wallRoughness = np.mean([1.11, 2.17])
hExterior = convectionDOE2(np.mean([1, 3]), windSpeed, wallRoughness) #using DOE-2 to calculate this

In [ ]:
runMyBEM(
    dataSampled,
    materials,
    floorTempAdjustment,
    hInterior,
    hExterior,
    alphaRoof,
    verbose = True,
    makePlots = True)